In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

26038


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計
26033,2021-07-18,47,沖縄県,70,21812,0,215
26034,2021-07-19,47,沖縄県,35,21847,5,220
26035,2021-07-20,47,沖縄県,154,22001,0,220
26036,2021-07-21,47,沖縄県,169,22170,4,224
26037,2021-07-22,47,沖縄県,153,22323,0,224


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計
42,静岡県,12188,10028,154
43,香川県,20498,2147,32
44,高知県,21606,1980,29
45,鳥取県,17174,626,2
46,鹿児島県,25484,3798,39


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数
26031,2021-07-16,47,沖縄県,76,21661,0,215,406.0
26032,2021-07-17,47,沖縄県,81,21742,0,215,423.0
26033,2021-07-18,47,沖縄県,70,21812,0,215,463.0
26034,2021-07-19,47,沖縄県,35,21847,5,220,470.0
26035,2021-07-20,47,沖縄県,154,22001,0,220,559.0
26036,2021-07-21,47,沖縄県,169,22170,4,224,645.0
26037,2021-07-22,47,沖縄県,153,22323,0,224,738.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)

pref.tail(10)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
26028,2021-07-13,47,沖縄県,65,21442,0,206,335.0,1.321429,0.142501,0.048387
26029,2021-07-14,47,沖縄県,83,21525,5,211,360.0,0.276923,0.191278,0.431034
26030,2021-07-15,47,沖縄県,60,21585,4,215,385.0,-0.277108,0.208342,0.714286
26031,2021-07-16,47,沖縄県,76,21661,0,215,406.0,0.266667,0.164804,0.381818
26032,2021-07-17,47,沖縄県,81,21742,0,215,423.0,0.065789,0.150826,0.265625
26033,2021-07-18,47,沖縄県,70,21812,0,215,463.0,-0.135802,0.207319,1.333333
26034,2021-07-19,47,沖縄県,35,21847,5,220,470.0,-0.500000,0.145414,0.250000
26035,2021-07-20,47,沖縄県,154,22001,0,220,559.0,3.400000,0.442353,1.369231
26036,2021-07-21,47,沖縄県,169,22170,4,224,645.0,0.097403,0.416707,1.036145
26037,2021-07-22,47,沖縄県,153,22323,0,224,738.0,-0.094675,0.442769,1.550000


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["日付","都道府県名","各地の感染者数_1日ごとの発表数","各地の死者数_1日ごとの発表数","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均","新規感染者数先週比"]]
latest.head()

,日付,都道府県名,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
553,2021-07-22,北海道,140,1,723.0,0.186441,0.114902,0.647059
1107,2021-07-22,青森県,4,0,45.0,0.333333,NaN,-0.428571
1661,2021-07-22,岩手県,5,0,53.0,-0.444444,0.080952,-0.166667
2215,2021-07-22,宮城県,18,0,178.0,-0.357143,-0.032313,-0.600000
2769,2021-07-22,秋田県,2,0,31.0,-0.714286,NaN,-0.600000


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
0,三重県,13296,5539,113,2021-07-22,27,0,137.0,0.227273,0.356019,1.454545
1,京都府,14404,17288,249,2021-07-22,71,0,360.0,-0.112500,0.198418,1.218750
2,佐賀県,22714,2629,24,2021-07-22,8,0,48.0,0.142857,0.200964,0.333333
3,兵庫県,15512,42255,1315,2021-07-22,149,0,715.0,0.241667,0.310352,0.935065
4,北海道,554,42818,1419,2021-07-22,140,1,723.0,0.186441,0.114902,0.647059


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口
0,三重県,13296,5539,113,2021-07-22,27,0,137.0,0.227273,0.356019,1.454545,1813859
1,京都府,14404,17288,249,2021-07-22,71,0,360.0,-0.112500,0.198418,1.218750,2545899
2,佐賀県,22714,2629,24,2021-07-22,8,0,48.0,0.142857,0.200964,0.333333,823810
3,兵庫県,15512,42255,1315,2021-07-22,149,0,715.0,0.241667,0.310352,0.935065,5549568
4,北海道,554,42818,1419,2021-07-22,140,1,723.0,0.186441,0.114902,0.647059,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,554,42818,1419,2021-07-22,140,1,723.0,0.186441,0.114902,0.647059,5267762,81.283095,812.830952,13.724994
41,青森県,1108,2606,31,2021-07-22,4,0,45.0,0.333333,NaN,-0.428571,1275783,20.426671,204.266713,3.527246
19,岩手県,1662,1879,47,2021-07-22,5,0,53.0,-0.444444,0.080952,-0.166667,1235517,15.208208,152.082084,4.289702
11,宮城県,2216,9626,91,2021-07-22,18,0,178.0,-0.357143,-0.032313,-0.600000,2292385,41.991201,419.912013,7.764839
36,秋田県,2770,986,20,2021-07-22,2,0,31.0,-0.714286,NaN,-0.600000,985416,10.005926,100.059264,3.145880
15,山形県,3324,2091,47,2021-07-22,4,0,16.0,1.000000,NaN,0.333333,1082296,19.320038,193.200381,1.478339
35,福島県,3878,5243,162,2021-07-22,14,0,102.0,-0.391304,0.123551,-0.176471,1881981,27.858942,278.589423,5.419821
38,茨城県,4432,11364,166,2021-07-22,86,0,386.0,0.116883,0.161221,0.791667,2921436,38.898679,388.986786,13.212680
27,栃木県,4986,7515,84,2021-07-22,33,0,204.0,-0.214286,0.395443,1.538462,1965516,38.234235,382.342347,10.378954
37,群馬県,5540,8216,155,2021-07-22,16,0,103.0,-0.542857,0.320243,1.285714,1969439,41.717464,417.174637,5.229916


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count        47.000000
mean      18178.106383
std       34301.328236
min         626.000000
25%        2617.500000
50%        5243.000000
75%       11564.000000
max      195041.000000
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,7202,195041,2276,2021-07-22,1979,0,9614.0,0.080240,0.118017,0.512997,13834925,140.977273,1409.772731,69.490800
1,大阪府,14958,108235,2716,2021-07-22,461,1,2385.0,-0.061100,0.104194,0.422840,8849635,122.304479,1223.044792,26.950264
2,神奈川県,7756,75109,980,2021-07-22,631,1,3442.0,0.211132,0.075663,0.565757,9209442,81.556516,815.565156,37.374686
3,愛知県,12742,52572,1002,2021-07-22,146,1,619.0,0.339450,0.251137,0.972973,7575530,69.397125,693.971247,8.171045
4,埼玉県,6094,51083,845,2021-07-22,510,0,2299.0,0.338583,0.100920,0.554878,7390054,69.123987,691.239875,31.109380
5,千葉県,6648,44637,728,2021-07-22,343,0,1853.0,0.135762,0.063107,0.355731,6319772,70.630713,706.307126,29.320678
6,北海道,554,42818,1419,2021-07-22,140,1,723.0,0.186441,0.114902,0.647059,5267762,81.283095,812.830952,13.724994
7,兵庫県,15512,42255,1315,2021-07-22,149,0,715.0,0.241667,0.310352,0.935065,5549568,76.141062,761.410618,12.883886
8,福岡県,22160,36797,528,2021-07-22,139,0,609.0,0.022059,0.179546,0.985714,5129841,71.731268,717.312681,11.871713
9,沖縄県,26038,22323,224,2021-07-22,153,0,738.0,-0.094675,0.442769,1.550000,1481547,150.673586,1506.735865,49.812797


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count     47.000000
mean      43.257082
std       32.018176
min        9.494733
25%       23.238411
50%       30.793178
75%       54.818181
max      150.673586
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
26,東京都,7202,195041,2276,2021-07-22,1979,0,9614.0,0.080240,0.118017,0.512997,13834925,140.977273,1409.772731,69.490800
28,沖縄県,26038,22323,224,2021-07-22,153,0,738.0,-0.094675,0.442769,1.550000,1481547,150.673586,1506.735865,49.812797
32,神奈川県,7756,75109,980,2021-07-22,631,1,3442.0,0.211132,0.075663,0.565757,9209442,81.556516,815.565156,37.374686
7,埼玉県,6094,51083,845,2021-07-22,510,0,2299.0,0.338583,0.100920,0.554878,7390054,69.123987,691.239875,31.109380
5,千葉県,6648,44637,728,2021-07-22,343,0,1853.0,0.135762,0.063107,0.355731,6319772,70.630713,706.307126,29.320678
9,大阪府,14958,108235,2716,2021-07-22,461,1,2385.0,-0.061100,0.104194,0.422840,8849635,122.304479,1223.044792,26.950264
31,石川県,9418,4388,116,2021-07-22,62,0,273.0,-0.060606,0.324306,1.952381,1139612,38.504333,385.043331,23.955522
45,鳥取県,17174,626,2,2021-07-22,11,0,105.0,-0.352941,0.178142,0.100000,561175,11.155166,111.551655,18.710741
1,京都府,14404,17288,249,2021-07-22,71,0,360.0,-0.112500,0.198418,1.218750,2545899,67.905286,679.052861,14.140388
4,北海道,554,42818,1419,2021-07-22,140,1,723.0,0.186441,0.114902,0.647059,5267762,81.283095,812.830952,13.724994


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    40.000000
mean      0.333795
std       0.380340
min      -0.032313
25%       0.117238
50%       0.226050
75%       0.369269
max       2.205467
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
17,岐阜県,11634,9416,187,2021-07-22,14,0,70.0,0.272727,2.205467,2.500000,2032490,46.327411,463.274112,3.444051
13,富山県,8864,2123,38,2021-07-22,12,0,51.0,0.333333,0.907143,2.000000,1055999,20.104186,201.041857,4.829550
30,熊本県,23822,6562,121,2021-07-22,14,0,54.0,0.400000,0.869048,3.666667,1769880,37.075960,370.759600,3.051054
16,山梨県,10526,2218,21,2021-07-22,14,0,42.0,1.000000,0.847619,13.000000,826579,26.833491,268.334908,5.081184
20,島根県,17728,645,2,2021-07-22,20,0,80.0,0.176471,0.674587,9.000000,679324,9.494733,94.947330,11.776413
40,長野県,11080,5114,94,2021-07-22,8,0,43.0,-0.111111,0.579365,7.000000,2087307,24.500469,245.004688,2.060071
6,和歌山県,16620,2769,49,2021-07-22,10,0,47.0,0.250000,0.464286,0.666667,954258,29.017310,290.173098,4.925293
28,沖縄県,26038,22323,224,2021-07-22,153,0,738.0,-0.094675,0.442769,1.550000,1481547,150.673586,1506.735865,49.812797
25,新潟県,8310,3682,54,2021-07-22,37,0,148.0,0.541667,0.438468,0.947368,2236042,16.466596,164.665959,6.618838
27,栃木県,4986,7515,84,2021-07-22,33,0,204.0,-0.214286,0.395443,1.538462,1965516,38.234235,382.342347,10.378954


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
11,宮城県,2216,9626,91,2021-07-22,18,0,178.0,-0.357143,-0.032313,-0.600000,2292385,41.991201,419.912013,7.764839
39,長崎県,23268,3308,69,2021-07-22,9,0,64.0,0.125000,0.047222,0.125000,1350769,24.489754,244.897536,4.738042
5,千葉県,6648,44637,728,2021-07-22,343,0,1853.0,0.135762,0.063107,0.355731,6319772,70.630713,706.307126,29.320678
23,愛媛県,21052,2813,75,2021-07-22,5,0,36.0,-0.375000,0.064744,-0.615385,1369131,20.545879,205.458791,2.629405
44,高知県,21606,1980,29,2021-07-22,7,0,57.0,0.400000,0.071712,-0.222222,709230,27.917601,279.176008,8.036885
32,神奈川県,7756,75109,980,2021-07-22,631,1,3442.0,0.211132,0.075663,0.565757,9209442,81.556516,815.565156,37.374686
19,岩手県,1662,1879,47,2021-07-22,5,0,53.0,-0.444444,0.080952,-0.166667,1235517,15.208208,152.082084,4.289702
7,埼玉県,6094,51083,845,2021-07-22,510,0,2299.0,0.338583,0.100920,0.554878,7390054,69.123987,691.239875,31.109380
9,大阪府,14958,108235,2716,2021-07-22,461,1,2385.0,-0.061100,0.104194,0.422840,8849635,122.304479,1223.044792,26.950264
4,北海道,554,42818,1419,2021-07-22,140,1,723.0,0.186441,0.114902,0.647059,5267762,81.283095,812.830952,13.724994


## グラフ化

In [19]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,5))
plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("各地の新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("1万人当たりの新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [22]:
fig = plt.figure(figsize=(9.5,10))

#plt.style.use("ggplot")
### 最新の各地の新規感染者数
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("{}の各地の新規感染者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の感染者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の感染者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 最新の各地の死者数
plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("{}の各地の新規死者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の死者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の死者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")



### 直近1週間の10万人当たりの新規感染者数
plt.subplot(2,2,3) # (rows, columns, panel number)
plt.title("直近1週間の10万人当たりの新規感染者数")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
plt.plot([15,15],[0,47],"--",lw=0.5,color="black")
plt.text(15,0,"Stage3",ha="center",va="top",fontsize=8)
plt.plot([25,25],[0,47],"--",lw=0.5,color="black")
plt.text(25,0,"Stage4",ha="center",va="top",fontsize=8)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 新規感染者数の増加速度（新規感染者数前週同曜日比）
plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("新規感染者数前週同曜日比")
chart_data = all_jp.sort_values("新規感染者数先週比",na_position="first")
plt.barh(chart_data["都道府県名"], chart_data["新規感染者数先週比"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [23]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [24]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[38  4  2  1  1  0  0  1]
[   626.     24927.875  49229.75   73531.625  97833.5   122135.375
 146437.25  170739.125 195041.   ]


## ヒストグラム

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([37.,  2.,  6.,  0.,  1.,  0.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <a list of 7 Patch objects>)

In [26]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([ 7., 22.,  6.,  7.,  2.,  0.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <a list of 6 Patch objects>)

In [27]:
pref[pref["都道府県名"]=="山口県"].tail(10)


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
19380,2021-07-13,35,山口県,2,3177,0,80,15.0,inf,NaN,0.000000
19381,2021-07-14,35,山口県,3,3180,0,80,17.0,0.500000,NaN,2.000000
19382,2021-07-15,35,山口県,1,3181,0,80,12.0,-0.666667,NaN,-0.833333
19383,2021-07-16,35,山口県,4,3185,0,80,14.0,3.000000,NaN,1.000000
19384,2021-07-17,35,山口県,6,3191,0,80,18.0,0.500000,NaN,2.000000
19385,2021-07-18,35,山口県,2,3193,0,80,18.0,-0.666667,NaN,0.000000
19386,2021-07-19,35,山口県,0,3193,0,80,18.0,-1.000000,NaN,NaN
19387,2021-07-20,35,山口県,3,3196,0,80,19.0,inf,NaN,0.500000
19388,2021-07-21,35,山口県,4,3200,0,80,20.0,0.333333,NaN,0.333333
19389,2021-07-22,35,山口県,4,3204,0,80,23.0,0.000000,NaN,3.000000
